# Python-daemon 库介绍


库的官方地址：https://pagure.io/python-daemon




## 守护进程的几个参数


守护进程的上下文参数：

1. **files_preserve**: 文件列表，当启动daemon时，这些文件不会被关闭。 列表的元素应该是文件描述符（file descriptors，例如file对象的fileno()方法的返回值）或者是Python的file对象。 
    - 如果设定为None（默认值），所有打开的文件将被关闭。
    
2. **chroot_directory**：进程根目录的完整路径，如果设为None（默认值），指定的根目录与父进程一样。 

3. **working_directory**：进程启动时的工作目录完整路径。默认为'/'根目录。如果进程它自己的当前工作目录所在的文件系统无法加载，那么该参数将设定为默认值或运行进程的home目录。 （具体如何，需要进一步调试）

4. **umask**：默认值为‘0’，进程创建文件时的umask值。该掩码控制如何为新创建的文件设置文件权限。通常，一个守护进程应该不依赖父进程的umask值，因为守护进程可能会超出了父进程的控制范围，父进程的umask值可能会阻止创建具有所需访问模式的文件。因此当守护进程启动（open），应设定一个明确的umask值。 但是，默认值‘0’设定的权限有点太大（umask(0)开放了所有的权限），可以考虑设定：0o022, 0o027, 0o077，或其他特定值。总之，确保守护进程能按访问权限的要求创建所有的文件。 
    - 三类用户：文件所有者user(u)，文件所有者所在组group(g)、其它用户others(o)
    - 三种权限：读read(r)、写write(w)、执行execute(x)。
    - 文件夹的基本码是rwxrwxrwx(777)，文件的基本码是rw-rw-rw-(666)
    - 其中前三位指明了所有者的权限、中间三位指明了组权限、最后三位指明了其它用户的权限。
    - umask在英文中是屏蔽的意思，那么umask=022，指屏蔽文件的022权限，到底屏蔽了什么？    
    - 022：表示默认创建新文件权限为755 也就是rxwr--r--(所有者全部权限，属组读，其它人读)
    - 027：表示默认创建新文件权限为750 也就是rxwr-x---(所有者全部权限，属组读，其它人无)
    - 077：表示默认创建新文件权限为700 也就是rxw------(所有者全部权限，属组无，其它人无)

5. **pidfile**：默认为None，PID锁文件。 当守护进程启动和关闭时，pidfile会进入和离开。

6. **detach_process**：默认为None，是否分离进程。如果为True，启动进程的时候会与父进程分离；为False，将不会分离。 
    - 如果未指定（None),初始化时将被默认为True。
    - 除非分离进程是冗余的，例如守护进程是被 `init`，`initd`或`inetd`启动。
    
7. **ignal_map**：默认是依赖操作系统，用于将回调操作与操作系统信号的映射。当守护进程启动后，该映射将会生效，并确定每个信号的回调。
    - None值：将忽略该信号（将信号响应设定为signal.SIG_IGN）
    - str值：可设定为DaemonContext的属性名称，该属性名称应该是信号对应的回调函数名称。 
    - 其他可用于信号回调的值：参照“signal.signal”文档了解更多的signal handler接口
    - 默认值依赖运行的操作系统，会按照下表进行默认设定：
        - ``signal.SIGTTIN``: ``None``
        - ``signal.SIGTTOU``: ``None``
        - ``signal.SIGTSTP``: ``None``
        - ``signal.SIGTERM``: ``'terminate'``
     - 如何设定回调处理，取决于如何与子进程交互，可能需要指定一个包含``signal.SIGCHLD``（当子进程退出时，收到该信号）的signal map。    
     
8. **uid**： 默认为os.getuid()返回值
9. gid: 默认为os.getgid()返回值：如果使用默认值(也就是，进程的实际UID和GID)，那么就将不会从父进程继承任何权限。
 
10. initgroups：默认为Flase。如果为True，守护进程的（supplementary groups）增补组为参数“uid”的所属组。注意：需要当前进程有更改进程所属组的权限。

11. prevent_core：默认为True。如果为True，则防止生成核心（core）文件，避免以root用户运行的守护进程泄漏敏感信息。

12. stdin、stdout和stderr ：默认为None。 
    - file-like对象，重新向到`sys.stdin`, `sys.stdout`和 `sys.stderr` 。对象需open，并且stdin有r权限，stdout和stderr有w+权限。 
    - 如果对象，有返回file描述符的`fileno()`方法，在守护进程启动期间，相应文件将不被关闭(它将被视为“文件保存”中列出的文件)。
    - 如果设定为None，将定向到os.devnull。
    - 说明：在类Unix系统中，/dev/null，或称空设备，是一个特殊的设备文件，它丢弃一切写入其中的数据（但报告写入操作成功），读取它则会立即得到一个EOF。空设备通常被用于丢弃不需要的输出流，或作为用于输入流的空文件。这些操作通常由重定向完成。

# 使用实例 1 

最简单的实例 

注意：daemon的实例，无法在jupyter进行运行，因为它会强制关闭进程，即使detach_process=False。 

In [ ]:
%load ./examples/simplest.py

In [ ]:
import daemon
import logging
import time
import signal
import threading

logfilename = '/home/xiaopeng/tmp/testdaemon.log'
logger = logging.getLogger("DaemonLog")
logger.setLevel(logging.INFO)
formatter = logging.Formatter(
    '%(asctime)s:%(levelname)s:%(message)s',
    '%Y-%m-%d %H:%M:%S')
handler = logging.FileHandler(logfilename)
handler.setFormatter(formatter)
logger.addHandler(handler)

def initial_program_setup():
    logger.info('daemon started')

def do_main_program():
    print("start the main program...")
    #while True:
    #    time.sleep(1)
    #    logger.info('another second passed')

def program_cleanup(signum, frame):
    logger.info('daemon stops')
    context.terminate(signum, frame)

def reload_program_config(signum, frame):
    logger.info('reloading config')
context = daemon.DaemonContext()

context.signal_map = {
    signal.SIGTERM: program_cleanup,
    signal.SIGHUP: 'terminate',
    signal.SIGUSR1: reload_program_config,
    }

context.files_preserve = [handler.stream]

initial_program_setup()

with context:
    print("ehl")
#    do_main_program()

print("end ")

## 附录 PEP 3143
